In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
import evaluate
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer
from transformers import RobertaForSequenceClassification
import torch
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [9]:
df = pd.read_csv('../data/raw/wikipedia_xs.csv')
df["label"] = df["category"].astype('category').cat.codes
df

,text,category,url,title,label
0,Think of Lampedusa is a collection of serial p...,Literature&Languages,https://en.wikipedia.org/wiki?curid=55818555,Think of Lampedusa,10
1,Michel DeGraff DeGraff holds a PhD in computer...,Social_sciences,https://en.wikipedia.org/wiki?curid=44947261,Michel DeGraff,15
2,"Experimenter's regress In science, experimente...",Psychology,https://en.wikipedia.org/wiki?curid=1608493,Experimenter's regress,13
3,"The Grand (Baltimore, Maryland) It was Sperry ...",Civil_engineering,https://en.wikipedia.org/wiki?curid=18162747,"The Grand (Baltimore, Maryland)",3
4,Rio de Janeiro has the second largest municipa...,History&Archaeology,https://en.wikipedia.org/wiki?curid=25936,Rio de Janeiro,7
...,...,...,...,...,...
295,Positivism Just as some critical theorists see...,Humanities,https://en.wikipedia.org/wiki?curid=2871407,Positivism,8
296,Thomas precession He thus found the missing fa...,Chemistry,https://en.wikipedia.org/wiki?curid=1868229,Thomas precession,2
297,"Abhi Golhar In May 2019, Think Realty launched...",Engineering&Technology,https://en.wikipedia.org/wiki?curid=56671795,Abhi Golhar,6
298,Auger electron spectroscopy Since the Auger ef...,Chemistry,https://en.wikipedia.org/wiki?curid=36835,Auger electron spectroscopy,2


In [10]:
# Split DataFrame into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df

,text,category,url,title,label
232,Flashback (psychology) This distinction dictat...,Biology,https://en.wikipedia.org/wiki?curid=2060821,Flashback (psychology),0
59,English land law An owner of land could direct...,Law,https://en.wikipedia.org/wiki?curid=31761911,English land law,9
6,Loudspeaker Some horn designs not only fold th...,Engineering&Technology,https://en.wikipedia.org/wiki?curid=45871,Loudspeaker,6
185,"Pramansagar ""Muni Pramansagar"" has authored se...",Religion&Philosophy&Ethics,https://en.wikipedia.org/wiki?curid=47845509,Pramansagar,14
173,History of the center of the Universe The cent...,Psychology,https://en.wikipedia.org/wiki?curid=35223708,History of the center of the Universe,13
...,...,...,...,...,...
188,Scandinavian design is a design movement chara...,Engineering&Technology,https://en.wikipedia.org/wiki?curid=6587771,Scandinavian design,6
71,Neuronal memory allocation It has been propose...,Biology,https://en.wikipedia.org/wiki?curid=35502244,Neuronal memory allocation,0
106,List of SIMILE projects Fresnel is a vocabular...,Engineering&Technology,https://en.wikipedia.org/wiki?curid=34229832,List of SIMILE projects,6
270,"Letters of Charles Lamb "" In the later letters...",Humanities,https://en.wikipedia.org/wiki?curid=35585812,Letters of Charles Lamb,8


In [11]:
from transformers import RobertaTokenizer

# Load the RoBERTa tokenizer
MODEL_NAME = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

# Tokenize function for DataFrame
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

In [12]:
from datasets import Dataset

# Convert train and test DataFrames to Hugging Face Dataset format
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)

# Tokenize the datasets
train_data = train_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

# Check the tokenized train data
print(train_data[0])

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

{'text': 'Flashback (psychology) This distinction dictates the manner in which memories are later recalled, namely either consciously (voluntarily) or unconsciously (involuntarily). These methods have largely relied on subtractive reasoning, in which the participant voluntarily recalls a memory and then the memory is again recalled, but this time through involuntary means. Involuntary memories (or flashbacks) are elicited in the participant by reading an emotionally charged script to them that is designed to trigger a flashback in individuals who suffer from PTSD. The investigators record the regions of the brain that are active during each of these conditions, and then subtract the activity. Whatever is left is assumed to underpin the neurological differences between the conditions. Imaging studies looking at patients with PTSD as they undergo flashback experiences have identified elevated activation in regions of the dorsal stream including the mid-occipital lobe, primary motor corte

In [18]:
print(train_data[0])

{'text': 'Flashback (psychology) This distinction dictates the manner in which memories are later recalled, namely either consciously (voluntarily) or unconsciously (involuntarily). These methods have largely relied on subtractive reasoning, in which the participant voluntarily recalls a memory and then the memory is again recalled, but this time through involuntary means. Involuntary memories (or flashbacks) are elicited in the participant by reading an emotionally charged script to them that is designed to trigger a flashback in individuals who suffer from PTSD. The investigators record the regions of the brain that are active during each of these conditions, and then subtract the activity. Whatever is left is assumed to underpin the neurological differences between the conditions. Imaging studies looking at patients with PTSD as they undergo flashback experiences have identified elevated activation in regions of the dorsal stream including the mid-occipital lobe, primary motor corte

In [4]:
precision_metric = evaluate.load("precision")
accuracy_metric = evaluate.load("accuracy")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    # Calculate metrics
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")

    # Combine and return metrics
    return {"accuracy": accuracy["accuracy"], "f1": f1["f1"]}

In [7]:
# Load RoBERTa model for sequence classification
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=16)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # Directory to save results
    eval_strategy="epoch",    # Evaluate every epoch
    learning_rate=2e-5,             # Smaller learning rate for fine-tuning
    per_device_train_batch_size=8,  # Batch size (adjust for hardware constraints)
    per_device_eval_batch_size=16,
    num_train_epochs=3,             # Number of training epochs
    weight_decay=0.01,              # Regularization
    logging_dir="./logs",           # Directory to save logs
    logging_steps=10,
    save_strategy="epoch",          # Save checkpoint every epoch
    load_best_model_at_end=True,    # Load the best checkpoint on validation loss
    metric_for_best_model="f1",     # Load best model based on F1 score
    
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics,  # Pass updated metrics
)

# Train the model
trainer.train()

  0%|          | 0/90 [00:00<?, ?it/s]

{'loss': 2.7404, 'grad_norm': 4.213046550750732, 'learning_rate': 1.7777777777777777e-05, 'epoch': 0.33}
{'loss': 2.7531, 'grad_norm': 6.526999473571777, 'learning_rate': 1.555555555555556e-05, 'epoch': 0.67}
{'loss': 2.7116, 'grad_norm': 3.162813186645508, 'learning_rate': 1.3333333333333333e-05, 'epoch': 1.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 2.704223394393921, 'eval_accuracy': 0.1, 'eval_f1': 0.01818181818181818, 'eval_runtime': 9.6055, 'eval_samples_per_second': 6.246, 'eval_steps_per_second': 0.416, 'epoch': 1.0}
{'loss': 2.5872, 'grad_norm': 6.99885368347168, 'learning_rate': 1.1111111111111113e-05, 'epoch': 1.33}
{'loss': 2.7104, 'grad_norm': 6.958769798278809, 'learning_rate': 8.888888888888888e-06, 'epoch': 1.67}
{'loss': 2.6732, 'grad_norm': 8.131162643432617, 'learning_rate': 6.666666666666667e-06, 'epoch': 2.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 2.635181427001953, 'eval_accuracy': 0.1, 'eval_f1': 0.0383318544809228, 'eval_runtime': 14.8569, 'eval_samples_per_second': 4.039, 'eval_steps_per_second': 0.269, 'epoch': 2.0}
{'loss': 2.5358, 'grad_norm': 9.84009075164795, 'learning_rate': 4.444444444444444e-06, 'epoch': 2.33}
{'loss': 2.5584, 'grad_norm': 10.769404411315918, 'learning_rate': 2.222222222222222e-06, 'epoch': 2.67}
{'loss': 2.4872, 'grad_norm': 8.786731719970703, 'learning_rate': 0.0, 'epoch': 3.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 2.5815024375915527, 'eval_accuracy': 0.1, 'eval_f1': 0.03688853247794707, 'eval_runtime': 12.6115, 'eval_samples_per_second': 4.758, 'eval_steps_per_second': 0.317, 'epoch': 3.0}
{'train_runtime': 453.5941, 'train_samples_per_second': 1.587, 'train_steps_per_second': 0.198, 'train_loss': 2.639719009399414, 'epoch': 3.0}


TrainOutput(global_step=90, training_loss=2.639719009399414, metrics={'train_runtime': 453.5941, 'train_samples_per_second': 1.587, 'train_steps_per_second': 0.198, 'total_flos': 189463772528640.0, 'train_loss': 2.639719009399414, 'epoch': 3.0})

In [14]:
# Evaluate on the test dataset
trainer.evaluate()

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 2.635181427001953,
 'eval_accuracy': 0.1,
 'eval_f1': 0.0383318544809228,
 'eval_runtime': 9.4525,
 'eval_samples_per_second': 6.348,
 'eval_steps_per_second': 0.423,
 'epoch': 3.0}

In [ ]:
# Move model to MPS device

device = torch.device("cpu")
model = model.to(device)

In [10]:
# Add a column for predictions in df
def predict(df, model, tokenizer):
    # Tokenize the text column
    encoded_inputs = tokenizer(
        df["text"].tolist(),
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    
    # Forward pass for predictions
    outputs = model(**encoded_inputs)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).numpy()
    
    # Add predictions to DataFrame
    return predicted_labels



In [11]:
# Apply prediction function to test_df
predicted_labels = predict(test_df, model, tokenizer)
classification_report(test_df["label"], predicted_labels)

: 

In [10]:
# Example text for inference
sample_text = ["I absolutely loved this movie!", "The product quality is disappointing."]

In [11]:
# Tokenize the sample text
encoded_inputs = tokenizer(sample_text, padding=True, truncation=True, return_tensors="pt")

# Forward pass for predictions
outputs = model(**encoded_inputs)
logits = outputs.logits

# Get predicted labels
import torch
predicted_labels = torch.argmax(logits, dim=1).numpy()
print(predicted_labels)

RuntimeError: Placeholder storage has not been allocated on MPS device!

In [12]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 6.5 MB/s eta 0:00:00a 0:00:01


In [13]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [15]:
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

Building jupyterlab assets (production, minimized)
